In [24]:
import numpy as np
import pandas as pd
import os

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Підготуємо датасет для кожного з видів активності і розрахуємо time domain features. Потім об’єднаємо всі датасети в один за допомогою методу pd.concat()

In [25]:
file_names=os.listdir('ds_notebooks/data/idle')
ds_idle=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"ds_notebooks/data/idle/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['idle']

      }
)

  ds_idle=pd.concat([ds_idle,td]).reset_index(drop=True)
ds_idle

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,5.099650,-0.909797,0.065840,4.616021,2,0.566011,-0.131681,3.647470,1.157603,0.167435,-2.365312,idle
1,0.407014,-0.320823,0.039505,0.023942,7,0.028411,-0.131681,0.568223,0.048059,0.131202,-2.409922,idle
2,-0.062249,-0.124498,0.021548,0.244209,14,0.012045,0.225055,-1.272920,0.016383,0.220905,-0.097477,idle
3,0.560243,0.268151,0.050279,0.086191,28,0.037882,-0.021548,-0.857829,0.054525,0.029528,0.565957,idle
4,0.560243,0.268151,0.051476,0.086191,26,0.036818,-0.019154,-1.001079,0.054004,0.027134,0.453139,idle
...,...,...,...,...,...,...,...,...,...,...,...,...
1034,0.488417,0.330400,0.045490,0.057461,15,0.026836,-0.028730,0.058832,0.037089,0.028411,0.337666,idle
1035,0.488417,0.330400,0.045490,0.057461,13,0.028092,-0.023942,-0.009364,0.038030,0.026815,0.428961,idle
1036,0.488417,0.320823,0.045490,0.057461,11,0.028730,-0.019154,-0.093211,0.038467,0.026336,0.426131,idle
1037,0.488417,0.320823,0.041899,0.057461,9,0.029528,-0.023942,-0.088563,0.038976,0.026017,0.393292,idle


In [26]:
file_names=os.listdir('ds_notebooks/data/running')
ds_run=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"ds_notebooks/data/running/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['running']

      }
)

  ds_run=pd.concat([ds_run,td]).reset_index(drop=True)
ds_run

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,22.720974,-4.941632,6.890514,19.091364,27,6.384644,6.210560,-0.685412,8.509691,5.481126,0.860710,running
1,26.499025,-10.994174,18.334796,26.015396,18,8.397519,8.166622,-0.781082,11.209556,7.044061,-0.093876,running
2,30.674513,-8.006210,8.998607,29.022512,29,12.953121,7.754819,-0.279669,15.659494,6.309839,-0.793851,running
3,22.261288,-8.547300,10.705673,32.671276,17,8.445701,6.560113,0.268587,11.718381,6.726430,1.510165,running
4,28.629864,-8.547300,11.465832,32.671276,15,8.942609,6.560113,0.280791,12.189022,7.104395,1.556591,running
...,...,...,...,...,...,...,...,...,...,...,...,...
3403,22.261288,-17.247830,13.491326,32.671276,27,7.477784,8.429984,-0.114744,10.418266,8.254409,0.689576,running
3404,22.261288,-17.247830,14.093468,32.671276,25,8.308199,8.760384,0.019250,11.351841,8.978576,0.656152,running
3405,22.261288,-17.247830,13.427880,32.671276,23,8.374492,7.117962,0.114864,11.622819,7.609571,0.646205,running
3406,22.261288,-12.282255,11.465832,32.671276,21,8.269413,7.117962,0.206350,11.474190,7.088274,1.598983,running


In [27]:
file_names=os.listdir('ds_notebooks/data/stairs')
ds_stairs=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"ds_notebooks/data/stairs/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['stairs']

      }
)

  ds_stairs=pd.concat([ds_stairs,td]).reset_index(drop=True)
ds_stairs

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,8.073248,-4.582502,3.848680,2.973599,19,3.314692,-7.153874,-0.280692,4.710145,7.161376,-0.473734,stairs
1,8.288726,-4.649539,5.097256,-0.483629,16,3.770602,-7.963115,-0.747500,4.791658,9.477846,-0.503615,stairs
2,8.398860,-6.224924,3.859453,-2.715025,3,3.063812,-9.289502,-0.968323,4.230074,9.368990,-1.608858,stairs
3,8.398860,-6.224924,3.518280,-2.715025,1,3.023143,-9.265560,-1.054399,4.164104,9.412564,-1.654556,stairs
4,8.398860,-6.224924,3.587712,-2.715025,4,2.736732,-8.482656,-0.318812,3.422683,8.606676,-1.845238,stairs
...,...,...,...,...,...,...,...,...,...,...,...,...
160,8.398860,-6.224924,4.224569,-2.715025,13,3.139596,-10.110713,-0.728449,4.482366,9.970413,-1.517546,stairs
161,8.398860,-6.224924,4.129998,-2.715025,11,3.129019,-9.837775,-0.861020,4.410401,9.944875,-1.644321,stairs
162,8.398860,-6.224924,3.991135,-2.715025,9,3.174754,-9.837775,-0.814399,4.417071,10.013190,-1.370589,stairs
163,8.398860,-6.224924,4.129998,-2.715025,7,3.073357,-10.110713,-0.765965,4.324824,10.231062,-1.290441,stairs


In [28]:
file_names=os.listdir('ds_notebooks/data/walking')
ds_walking=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"ds_notebooks/data/walking/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['walking']

      }
)

  ds_walking=pd.concat([ds_walking,td]).reset_index(drop=True)
ds_walking

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,12.363658,-2.389413,4.782418,4.668694,8,4.076527,-10.472238,0.659964,5.590863,9.385271,-1.635672,walking
1,7.527370,-9.504981,6.580464,3.423709,15,3.955604,-9.576807,0.153039,5.336400,9.752541,-0.099639,walking
2,1.278504,-14.159308,3.665522,5.214572,8,4.378910,-9.670181,0.815229,5.583739,8.372842,0.982072,walking
3,11.113884,-12.277467,6.676232,6.435614,8,4.839959,-10.307039,0.259736,6.368325,10.455798,2.074244,walking
4,11.113884,-12.277467,7.320272,6.435614,6,5.236045,-10.307039,0.113506,6.823270,10.837434,2.082041,walking
...,...,...,...,...,...,...,...,...,...,...,...,...
1845,7.192182,-5.990292,5.601235,0.181959,18,4.100150,-10.414777,-0.540493,5.285310,10.702081,1.221757,walking
1846,7.192182,-7.359776,5.035007,0.181959,16,4.176020,-10.414777,-0.523121,5.329992,10.736398,1.178148,walking
1847,11.113884,-7.359776,5.007473,0.181959,14,4.327631,-10.414777,-0.453662,5.412533,10.878295,1.996085,walking
1848,11.113884,-7.359776,4.416105,0.181959,12,4.135180,-10.747571,-0.365569,5.294201,11.146126,1.959435,walking


In [29]:
data=pd.concat([ds_idle, ds_run, ds_stairs, ds_walking]).reset_index(drop=True)
data

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,5.099650,-0.909797,0.065840,4.616021,2,0.566011,-0.131681,3.647470,1.157603,0.167435,-2.365312,idle
1,0.407014,-0.320823,0.039505,0.023942,7,0.028411,-0.131681,0.568223,0.048059,0.131202,-2.409922,idle
2,-0.062249,-0.124498,0.021548,0.244209,14,0.012045,0.225055,-1.272920,0.016383,0.220905,-0.097477,idle
3,0.560243,0.268151,0.050279,0.086191,28,0.037882,-0.021548,-0.857829,0.054525,0.029528,0.565957,idle
4,0.560243,0.268151,0.051476,0.086191,26,0.036818,-0.019154,-1.001079,0.054004,0.027134,0.453139,idle
...,...,...,...,...,...,...,...,...,...,...,...,...
6457,7.192182,-5.990292,5.601235,0.181959,18,4.100150,-10.414777,-0.540493,5.285310,10.702081,1.221757,walking
6458,7.192182,-7.359776,5.035007,0.181959,16,4.176020,-10.414777,-0.523121,5.329992,10.736398,1.178148,walking
6459,11.113884,-7.359776,5.007473,0.181959,14,4.327631,-10.414777,-0.453662,5.412533,10.878295,1.996085,walking
6460,11.113884,-7.359776,4.416105,0.181959,12,4.135180,-10.747571,-0.365569,5.294201,11.146126,1.959435,walking


Застосуємо алгоритми SVM та випадковий ліс з бібліотеки scikit-learn до нашого датасету. Метрики оцінки, такі як точність (Аccuracy) та F1-показник (F1 Score), допомагають зрозуміти, яка модель працює краще для даного датасету. Confusion matrix допоможе побачити, які класи добре або погано класифікуються кожною з моделей. Ми також використовуємо GridSearchCV для пошуку найкращих значень гіперпараметрів C для SVM та max_depth для випадкового лісу. Крос-валідація з параметром cv=5 допомагає уникнути перенавчання та підвищити надійність результатів.

In [30]:
X = data.drop("id", axis=1)
y = data["id"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_param_grid = {'C': [0.1, 1.0, 10.0, 100.0]}
svm_model = SVC()
svm_grid_search = GridSearchCV(svm_model, svm_param_grid, cv=5)
svm_grid_search.fit(X_train, y_train)

rf_param_grid = {'max_depth': [None, 10, 20, 30]}
rf_model = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

best_svm_model = svm_grid_search.best_estimator_
best_rf_model = rf_grid_search.best_estimator_

svm_preds = best_svm_model.predict(X_test)
rf_preds = best_rf_model.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

svm_f1 = f1_score(y_test, svm_preds, average='weighted')
rf_f1 = f1_score(y_test, rf_preds, average='weighted')

svm_confusion_matrix = confusion_matrix(y_test, svm_preds)
rf_confusion_matrix = confusion_matrix(y_test, rf_preds)

print("Best SVM Model Parameters:", svm_grid_search.best_params_)
print("Best Random Forest Model Parameters:", rf_grid_search.best_params_)

print("\nSVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("\nSVM F1 Score:", svm_f1)
print("Random Forest F1 Score:", rf_f1)

print("\nSVM Confusion Matrix:")
print(svm_confusion_matrix)

print("\nRandom Forest Confusion Matrix:")
print(rf_confusion_matrix)

Best SVM Model Parameters: {'C': 100.0}
Best Random Forest Model Parameters: {'max_depth': None}

SVM Accuracy: 0.9876256767208044
Random Forest Accuracy: 0.9976798143851509

SVM F1 Score: 0.9858626001895158
Random Forest F1 Score: 0.997623651775331

SVM Confusion Matrix:
[[220   0   0   0]
 [  0 689   0   0]
 [  0   0  15  15]
 [  0   0   1 353]]

Random Forest Confusion Matrix:
[[220   0   0   0]
 [  0 689   0   0]
 [  0   0  27   3]
 [  0   0   0 354]]


Як бачимо з отриманих результатів, обидва алгоритми досить непогано справилися із задачею класифікації видів активності та все ж Random Forest виявився на один відсоток точнішим за SVM. До того ж, як показує Confusion Matrix, обидві моделі мали певні проблеми з класифікацією такого виду активності як stairs. Але якщо SVM виявив 15 помилкових класифікацій, то випадковий ліс лише 3. Також SVM один раз помилився при класифікації walking. Загалом, результати показують, що випадковий ліс показує кращі результати в порівнянні з SVM на цьому конкретному датасеті, оскільки має вищі значення точності та F1-показника, а також меншу кількість помилкових класифікацій.